<a href="https://colab.research.google.com/github/Sugirjan/Fare-Classification/blob/master/ML_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
from scipy.stats import randint
from google.colab import files

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFE
from sklearn.feature_selection import RFECV

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/My Drive/MLAssignment/train.csv');
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17176 entries, 0 to 17175
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     17176 non-null  int64  
 1   additional_fare            16974 non-null  float64
 2   duration                   16974 non-null  float64
 3   meter_waiting              16974 non-null  float64
 4   meter_waiting_fare         16974 non-null  float64
 5   meter_waiting_till_pickup  16974 non-null  float64
 6   pickup_time                17176 non-null  object 
 7   drop_time                  17176 non-null  object 
 8   pick_lat                   17176 non-null  float64
 9   pick_lon                   17176 non-null  float64
 10  drop_lat                   17176 non-null  float64
 11  drop_lon                   17176 non-null  float64
 12  fare                       17039 non-null  float64
 13  label                      17176 non-null  obj

In [5]:
# Drop null values
data = df.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16968 entries, 0 to 17175
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     16968 non-null  int64  
 1   additional_fare            16968 non-null  float64
 2   duration                   16968 non-null  float64
 3   meter_waiting              16968 non-null  float64
 4   meter_waiting_fare         16968 non-null  float64
 5   meter_waiting_till_pickup  16968 non-null  float64
 6   pickup_time                16968 non-null  object 
 7   drop_time                  16968 non-null  object 
 8   pick_lat                   16968 non-null  float64
 9   pick_lon                   16968 non-null  float64
 10  drop_lat                   16968 non-null  float64
 11  drop_lon                   16968 non-null  float64
 12  fare                       16968 non-null  float64
 13  label                      16968 non-null  obj

In [6]:
data.pickup_time = pd.to_datetime(data.pickup_time)
data.drop_time = pd.to_datetime(data.drop_time)
data['time_diff'] = data.drop_time - data.pickup_time
data['time_diff'] = data['time_diff'].astype('timedelta64[s]')
data.shape

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

(16968, 15)

In [ ]:
import geopy.distance
data['dist_diff'] = 0

pla = np.array(data.pick_lat)
plo = np.array(data.pick_lon)
dla = np.array(data.drop_lat)
dlo = np.array(data.drop_lon)
t = []

for i,j,k,l in zip(pla,plo,dla,dlo):
  coords_1 = (i, j)
  coords_2 = (k, l)
  t.append(geopy.distance.distance(coords_1, coords_2).km)

data['dist_diff'] = t
data.head()

In [8]:
data_manipulated = data.copy().drop(['pickup_time', 'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon','tripid'], axis=1)
data_manipulated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16968 entries, 0 to 17175
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            16968 non-null  float64
 1   duration                   16968 non-null  float64
 2   meter_waiting              16968 non-null  float64
 3   meter_waiting_fare         16968 non-null  float64
 4   meter_waiting_till_pickup  16968 non-null  float64
 5   fare                       16968 non-null  float64
 6   label                      16968 non-null  object 
 7   time_diff                  16968 non-null  float64
 8   dist_diff                  16968 non-null  float64
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


Data visualization

In [ ]:
# for col in data_manipulated.columns:
#   if(col in ['tripid', 'label']): 
#     continue
#   print(col)
#   sns.boxplot(data_manipulated)
# sns.boxplot(data_manipulated.dist_diff)
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(data_manipulated.dist_diff, data_manipulated.fare)
ax.set_xlabel('Distance')
ax.set_ylabel('Fare')
plt.show()

In [ ]:
# Removed outliers
sns.boxplot(data_manipulated[data_manipulated.dist_diff < 100].dist_diff)
# data_manipulated[data_manipulated.dist_diff > 100].

In [ ]:
data_manipulated.drop(['label'], axis=1).plot(kind='box', figsize=(30,10))

In [ ]:
# Dropping the outlier rows with Percentiles
# data_manipulated_out = data_manipulated
# print(data_manipulated_out.shape)
# cols = ['A4', 'A8', 'A12', 'A16','A20','A24','A28', 'A32']
# for col in cols: 
#   upper_lim = features_data[col].quantile(.999)
#   lower_lim = features_data[col].quantile(.001)
#   features_data = features_data[(features_data[col] < upper_lim)]
#   # features_data = features_data[(features_data[col] > lower_lim)]
# print(features_data.shape)

In [ ]:
sns.scatterplot(data_manipulated.meter_waiting, data_manipulated.meter_waiting_fare, hue='label', data=data_manipulated)
# plt.show()

In [9]:
data_manipulated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16968 entries, 0 to 17175
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            16968 non-null  float64
 1   duration                   16968 non-null  float64
 2   meter_waiting              16968 non-null  float64
 3   meter_waiting_fare         16968 non-null  float64
 4   meter_waiting_till_pickup  16968 non-null  float64
 5   fare                       16968 non-null  float64
 6   label                      16968 non-null  object 
 7   time_diff                  16968 non-null  float64
 8   dist_diff                  16968 non-null  float64
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


In [10]:
X = data_manipulated.copy().drop('label', axis=1)

t = []

for i in data_manipulated.label:
  if(i == 'correct'):
    t.append(1)
  else:
    t.append(0)

data_manipulated['target'] = t
y = data_manipulated['target']
num_pipeline = Pipeline([('std_scaler', StandardScaler())])
X_tr = num_pipeline.fit_transform(X)
data_manipulated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16968 entries, 0 to 17175
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   additional_fare            16968 non-null  float64
 1   duration                   16968 non-null  float64
 2   meter_waiting              16968 non-null  float64
 3   meter_waiting_fare         16968 non-null  float64
 4   meter_waiting_till_pickup  16968 non-null  float64
 5   fare                       16968 non-null  float64
 6   label                      16968 non-null  object 
 7   time_diff                  16968 non-null  float64
 8   dist_diff                  16968 non-null  float64
 9   target                     16968 non-null  int64  
dtypes: float64(8), int64(1), object(1)
memory usage: 1.4+ MB


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_tr, y, test_size=0.2,random_state=42, stratify=y)
X_train.shape

(13574, 8)

Logistic Regression

In [ ]:
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test) 
logreg.score(X_test, y_test)

0.9180907483794932

SGD

In [ ]:
sgd = SGDClassifier(loss="hinge", penalty="elasticnet", max_iter=10)
sgd.fit(X_train, y_train)
y_pred = sgd.predict(X_test) 
sgd.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


0.9278137890394814

In [ ]:
# from sklearn.kernel_approximation import RBFSampler
# rbf_feature = RBFSampler(gamma=1, random_state=1)
# X_features = rbf_feature.fit_transform(X)
# X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2,random_state=42, stratify=y)
# sgd_rbf = SGDClassifier(loss="hinge", penalty="l2", max_iter=5)
# sgd_rbf.fit(X_train, y_train)
# y_pred = sgd_rbf.predict(X_test) 
# sgd_rbf.score(X_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


0.9101355332940483

In [ ]:
randomForestselect = SelectFromModel(RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1))
randomForestselect.fit(X_train, y_train)
X_train_rfc = randomForestselect.transform(X_train)
X_test_rfc = randomForestselect.transform(X_test)
rf = RandomForestClassifier(n_estimators=250, random_state=0, n_jobs=-1)
rf.fit(X_train_rfc,y_train)
rf.score(X_test_rfc, y_test)

0.952857984678845

In [ ]:
ls_select = SelectFromModel(LinearSVC(penalty="l1", dual=False))
ls_select.fit(X_train, y_train)
X_train_ls = ls_select.transform(X_train)
X_test_ls = ls_select.transform(X_test)
rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
rf.fit(X_train_ls,y_train)
print('with RF',rf.score(X_test_ls, y_test))
xtc = ExtraTreesClassifier(n_estimators=50)
xtc.fit(X_train_ls,y_train)
print('with XTC', xtc.score(X_test_ls, y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


with RF 0.9534472598703595
with XTC 0.9525633470830878


In [ ]:
xtc_select = SelectFromModel(ExtraTreesClassifier(n_estimators=50))
xtc_select.fit(X_train, y_train)
X_train_xtc = xtc_select.transform(X_train)
X_test_xtc = xtc_select.transform(X_test)
rf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
rf.fit(X_train_xtc,y_train)
print('with RF',rf.score(X_test_xtc, y_test))
xtc = ExtraTreesClassifier(n_estimators=50)
xtc.fit(X_train_xtc,y_train)
print('with XTC', xtc.score(X_test_xtc, y_test))

with RF 0.9540365350618739
with XTC 0.9499116087212728


In [ ]:
selector = RFECV(GradientBoostingClassifier(n_estimators=100, max_depth=32, random_state=0))
selector.fit(X_train, y_train)
X_train_Grad_RFE = selector.transform(X_train)
X_test_Grad_RFE = selector.transform(X_test)
gf = GradientBoostingClassifier(n_estimators=100, random_state=0)
gf.fit(X_train_Grad_RFE, y_train)
print('gf', gf.score(X_test_Grad_RFE, y_test))
rf = RandomForestClassifier(n_estimators=100,max_depth=32, random_state=0, n_jobs=-1)
rf.fit(X_train_Grad_RFE,y_train)
print('rf', rf.score(X_test_Grad_RFE, y_test))

gf 0.9478491455509723
rf 0.9543311726576311


In [ ]:
parameters = {
    "n_estimators":[100, 200, 300],
    "max_depth":[16,32,64,None],
}
rf1 = RandomForestClassifier()
cv_rf = GridSearchCV(rf1,parameters,cv=5)
cv_rf.fit(X_train, y_train)
print(cv_rf.score(X_test, y_test))
print("Tuned Decision Tree Parameters: {}".format(cv_rf.best_params_))

0.9537418974661167
Tuned Decision Tree Parameters: {'max_depth': 64, 'n_estimators': 300}


In [ ]:
selector = RFE(RandomForestClassifier(n_estimators=100, max_depth= 64, n_jobs=-1),step=1)
selector.fit(X_train, y_train)
X_train_Grad_RFELoop = selector.transform(X_train)
X_test_Grad_RFELoop = selector.transform(X_test)
rf = RandomForestClassifier(n_estimators=300, max_depth= 64, n_jobs=-1)
rf.fit(X_train_Grad_RFELoop,y_train)
print('rf', rf.score(X_test_Grad_RFELoop, y_test))

rf 0.9525633470830878


For submisssion

In [ ]:
test_submission = pd.read_csv('/content/drive/My Drive/MLAssignment/test.csv');
test_submission.shape

(8576, 13)

In [ ]:
def downloadTarget(target, fileName):
  submission = pd.DataFrame(target, columns=['prediction'])
  submission['tripid'] = test_submission.tripid
  submission.to_csv(fileName) 
  files.download(fileName)

In [ ]:
def feature_selection(data):
  data.pickup_time = pd.to_datetime(data.pickup_time)
  data.drop_time = pd.to_datetime(data.drop_time)
  data['time_diff'] = data.drop_time - data.pickup_time
  data['time_diff'] = data['time_diff'].astype('timedelta64[s]')

  data['dist_diff'] = 0

  pla = np.array(data.pick_lat)
  plo = np.array(data.pick_lon)
  dla = np.array(data.drop_lat)
  dlo = np.array(data.drop_lon)
  t = []

  for i,j,k,l in zip(pla,plo,dla,dlo):
    coords_1 = (i, j)
    coords_2 = (k, l)
    t.append(geopy.distance.distance(coords_1, coords_2).km)

  data['dist_diff'] = t

  data_manipulated = data.copy().drop(['pickup_time', 'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'tripid'], axis=1)
  return data_manipulated

In [ ]:
X.shape

(16968, 8)

In [ ]:
manipulated_data1 = feature_selection(test_submission)
X_test = manipulated_data1

sc = StandardScaler()
sc.fit(X)
X_tr = sc.transform(X)
X_test_tr = sc.transform(X_test)


In [ ]:
xtc_select = SelectFromModel(ExtraTreesClassifier(n_estimators=50))
xtc_select.fit(X_tr, y)
X_train_xtc = xtc_select.transform(X_tr)
xtc = ExtraTreesClassifier(n_estimators=50)
xtc.fit(X_train_xtc,y)
y_test_xtc = xtc.predict(xtc_select.transform(X_test_tr))
downloadTarget(y_test_xtc, 'submissionWithRFWithXTC-XTC.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>